In [50]:
api_key=""

In [2]:
import pandas as pd
data = {
    "transaction_id": ["T1001", "T1002", "T1003", "T1004", "T1005"],
    "customer_id": ["C001", "C002", "C003", "C002", "C001"],
    "payment_amount": [125.50, 89.99, 120.00, 54.30, 210.20],
    "payment_date": [
        "2021-10-05",
        "2021-10-06",
        "2021-10-07",
        "2021-10-05",
        "2021-10-08",
    ],
    "payment_status": ["Paid", "Unpaid", "Paid", "Paid", "Pending"],
}
df = pd.DataFrame(data)

In [3]:
df

,transaction_id,customer_id,payment_amount,payment_date,payment_status
0,T1001,C001,125.50,2021-10-05,Paid
1,T1002,C002,89.99,2021-10-06,Unpaid
2,T1003,C003,120.00,2021-10-07,Paid
3,T1004,C002,54.30,2021-10-05,Paid
4,T1005,C001,210.20,2021-10-08,Pending


In [4]:
data = """
    "transaction_id": ["T1001", "T1002", "T1003", "T1004", "T1005"],
    "customer_id": ["C001", "C002", "C003", "C002", "C001"],
    "payment_amount": [125.50, 89.99, 120.00, 54.30, 210.20],
    "payment_date": [
        "2021-10-05",
        "2021-10-06",
        "2021-10-07",
        "2021-10-05",
        "2021-10-08",
    ],
    "payment_status": ["Paid", "Unpaid", "Paid", "Paid", "Pending"],
}
"""
transaction_id = "T1001"

In [5]:
prompt = f"""
given the following data , what is status for transaction id {transaction_id}

data:
{data}
"""

In [8]:
import os
from mistralai.client import MistralClient 
from mistralai.models.chat_completion import ChatMessage


In [10]:
def mistral(user_message, model="mistral-small-latest", is_json=False):
    client = MistralClient(api_key=api_key)
    messages = [ChatMessage(role="user",content=user_message)]
    if is_json:
        chat_response = client.chat(
            model=model,messages=messages,response_format={"type": "json_object"}
        )
    else:
        chat_response = client.chat(model=model,messages=messages)
    return chat_response.choices[0].message.content

In [12]:
mistral(user_message=prompt)

'The status for transaction ID T1001 is "Paid".'

In [13]:
import json

In [14]:
def retrieve_payment_status(df:data,transaction_id:str) -> str:
    if transaction_id in df.transaction_id.values:
        return json.dumps(
            {"status":df[df.transaction_id == transaction_id].payment_status.item()}
        )
    return json.dumps({"error": "transaction id not found."})

In [16]:
retrieve_payment_status(df, transaction_id="T1002")

'{"status": "Unpaid"}'

In [17]:
def retrieve_payment_date(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values:
        return json.dumps(
            {"date": df[df.transaction_id == transaction_id].payment_date.item()}
        )
    return json.dumps({"error": "transaction id not found."})

In [18]:
tool_payment_status = {
    "type": "function",
    "function": {
        "name": "retrieve_payment_status",
        "description": "Get payment status of a transaction",
        "parameters": {
            "type": "object",
            "properties": {
                "transaction_id": {
                    "type": "string",
                    "description": "The transaction id.",
                }
            },
            "required": ["transaction_id"],
        },
    },
}

In [19]:
tool_payment_date = {
    "type": "function",
    "function": {
        "name": "retrieve_payment_date",
        "description": "Get payment date of a transaction",
        "parameters": {
            "type": "object",
            "properties": {
                "transaction_id": {
                    "type": "string",
                    "description": "The transaction id.",
                }
            },
            "required": ["transaction_id"],
        },
    },
}

In [20]:
tools = [tool_payment_status,tool_payment_date]

In [22]:
import functools

In [23]:
names_to_functions = {
    "retrieve_payment_status": functools.partial(retrieve_payment_status,df),
    "retrieve_payment_date": functools.partial(retrieve_payment_date,df)
}

In [24]:
names_to_functions["retrieve_payment_status"](transaction_id="T1001")

'{"status": "Paid"}'

In [25]:
tools

[{'type': 'function',
  'function': {'name': 'retrieve_payment_status',
   'description': 'Get payment status of a transaction',
   'parameters': {'type': 'object',
    'properties': {'transaction_id': {'type': 'string',
      'description': 'The transaction id.'}},
    'required': ['transaction_id']}}},
 {'type': 'function',
  'function': {'name': 'retrieve_payment_date',
   'description': 'Get payment date of a transaction',
   'parameters': {'type': 'object',
    'properties': {'transaction_id': {'type': 'string',
      'description': 'The transaction id.'}},
    'required': ['transaction_id']}}}]

In [26]:
from mistralai.models.chat_completion import ChatMessage
from mistralai.client import MistralClient

In [37]:
chat_history = [
    ChatMessage(role="user", content="What's the status of my transaction?")
]
model="mistral-small-latest"
client = MistralClient(api_key=api_key)
response = client.chat(
    model=model,messages=chat_history,tools=tools
)

In [38]:
response.choices[0].message.content

'To check the status of your transaction, I need the transaction ID. Once you provide it, I can use the "retrieve_payment_status" function to get the information for you.\n\nFor example, if your transaction ID is "abc123", here\'s how I would use the function:\n\n```json\n[{"name": "retrieve_payment_status", "arguments": {"transaction_id": "abc123"}}]\n```\n\nPlease provide the transaction ID, and I\'ll help you get the status.'

In [35]:
# chat_history = [
#     ChatMessage(role="user", content="What's the payment status of my transaction T1001 ?")
# ]
# model="mistral-small-latest"
# client = MistralClient(api_key=api_key)
# response = client.chat(
#     model=model,messages=chat_history,tools=tools
# ).choices[0].message.content

In [39]:
chat_history.append(
    ChatMessage(role="assistant", content=response.choices[0].message.content)
)
chat_history.append(ChatMessage(role="user", content="My transaction ID is T1001."))
chat_history

[ChatMessage(role='user', content="What's the status of my transaction?", name=None, tool_calls=None),
 ChatMessage(role='assistant', content='To check the status of your transaction, I need the transaction ID. Once you provide it, I can use the "retrieve_payment_status" function to get the information for you.\n\nFor example, if your transaction ID is "abc123", here\'s how I would use the function:\n\n```json\n[{"name": "retrieve_payment_status", "arguments": {"transaction_id": "abc123"}}]\n```\n\nPlease provide the transaction ID, and I\'ll help you get the status.', name=None, tool_calls=None),
 ChatMessage(role='user', content='My transaction ID is T1001.', name=None, tool_calls=None)]

In [40]:
response = client.chat(
    model=model, messages=chat_history, tools=tools, tool_choice="auto"
)

In [41]:
response.choices[0].message

ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='zJm74zUuP', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1001"}'))])

In [42]:
chat_history.append(response.choices[0].message)

In [43]:
tool_function = response.choices[0].message.tool_calls[0].function
print(tool_function)

name='retrieve_payment_status' arguments='{"transaction_id": "T1001"}'


In [44]:
args = json.loads(tool_function.arguments)
print(args)

{'transaction_id': 'T1001'}


In [45]:
function_result = names_to_functions[tool_function.name](**args)
function_result

'{"status": "Paid"}'

In [46]:
tool_msg = ChatMessage(role="tool", name=tool_function.name, content=function_result)
chat_history.append(tool_msg)

In [47]:
chat_history

[ChatMessage(role='user', content="What's the status of my transaction?", name=None, tool_calls=None),
 ChatMessage(role='assistant', content='To check the status of your transaction, I need the transaction ID. Once you provide it, I can use the "retrieve_payment_status" function to get the information for you.\n\nFor example, if your transaction ID is "abc123", here\'s how I would use the function:\n\n```json\n[{"name": "retrieve_payment_status", "arguments": {"transaction_id": "abc123"}}]\n```\n\nPlease provide the transaction ID, and I\'ll help you get the status.', name=None, tool_calls=None),
 ChatMessage(role='user', content='My transaction ID is T1001.', name=None, tool_calls=None),
 ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='zJm74zUuP', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1001"}'))]),
 ChatMessage(role='tool', content='{"status": "Paid"}', name='retrieve_pa

In [48]:
response = client.chat(model=model, messages=chat_history)
response.choices[0].message.content

'According to the information I retrieved, your transaction T1001 has a status of "Paid". Is there anything else you need assistance with?'